# Classification using pretrained, well-known models
This notebook aims to create a set of benchmarks for the project, using well-known, thoroughly studied models.
Either with pretrained weights, or training with new data.

## Notes
### 21.08.19
Attempting to classify with VGG has not proven effective yet.
Initially, the image data was note scaled at all. Implemented scaling in the
import function, using min-max scaling of the value. This preserves the inherent
intensity difference between images.

The number of layers of VGG16 used is varied between 3 to 9 without noticable
difference. Attempting to find out why, by analyzing the extracted features.
The idea is that in order to classify, the feature distribution should be
different for images containing single and double events.
I first attempt this with manual qualitative inspection.

Manual, qualitative inspection reveals that the distributions look very similar.
Performing a quantitative study using Kolmogorov-Smirnov two-sample test,
comparing the distribution for each feature.

* For 1 block (depth 3), the pvalue returned from comparisons is 1.0 for all features.
* For 2 blocks (depth 6), the pvalue returned from comparisons is 1.0 for all features.
* For 3 blocks (depth 10), the pvalue returned from comparisons is 1.0 for all features.
* For 4 blocks, (depth 14) the pvalue returned from comparisons is 1.0 for all features.
* For 5 blocks, (depth 18) the pvalue returned from comparisons is 1.0 for all features.

This indicates that extracting features using vgg16 doesn't work for classification.
I still want to confirm that the weights of the vgg layers are the imagenet weights.

### 22.08.19
Going to use a reference image from imagenet to verify that the vgg-layers behave
as expected.
* Reference produces very similar feature output as simulated data

Rewrote the vgg_model script to be able to import any pretrained model from
tensorflow, and extended data import to handle single files (for large file)
and possibility to specify number of samples to include.

### 23.08.19
Import scripts fixed so that array dimensions are correct independent of folder or single
file import.

Tests on multiple nets with 10k events give same results as for VGG.

### 24.08.19
Running checks on feature distribution with the full networks and 200k events.
With 200k events there are models which from the p-value given by the KS two-sample test
should be possible to classify with. Interstingly, VGG16 and VGG19 show a large variance in which
features are seemingly drawn from different distributions.

Need to run for: NASNetLarge, ResNet50 and Xception, but running into memory problems.

### 25.08.19
Some NaN values in output features from VGG. When removed, the network trains to acc = 0.9
on the features. 

TODO:
* store representation of data as files to avoid having to re-import
* Implement as two data functions. One for storing the data, one for importing. Should be doable with just numpy. 


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import sys
import matplotlib.pyplot as plt
from master_data_functions.functions import import_data
from master_models.pretrained import pretrained_model

# File import
# Sample filenames are:
# CeBr10kSingle_1.txt -> single events, 
# CeBr10kSingle_2.txt -> single events
# CeBr10k_1.txt -> mixed single and double events 
# CeBr10.txt -> small file of 10 samples
# CeBr2Mil_Mix.txt -> 2 million mixed samples of simulated events
    
folder = "simulated"
filename = "CeBr2Mil_Mix.txt"
num_samples = 2e5
#folder = "sample"
#filename = "CeBr10k_1.txt"
#num_samples = 1e3

data = import_data(folder=folder, filename=filename, num_samples=num_samples)
images = data[filename]["images"]
energies = data[filename]["energies"]
positions = data[filename]["positions"]
labels = to_categorical(data[filename]["labels"])
n_classes = labels.shape[1]


print("Number of classes: {}".format(n_classes))
print("Images shape: {}".format(images.shape))
print("Energies shape: {}".format(energies.shape))
print("Positions shape: {}".format(positions.shape))
print("Labels shape: {}".format(labels.shape))
      

/home/geir/git/master_analysis/notebooks/pretrained/pretrained_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/geir/git/master_analysis/notebooks/pretrained/pretrained_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/geir/git/master_analysis/notebooks/pretrained/pretrained_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'

Number of classes: 2
Images shape: (200000, 16, 16, 1)
Energies shape: (200000, 2)
Positions shape: (200000, 4)
Labels shape: (200000, 2)


In [2]:
# VGG16 expects 3 channels. Solving this by concatenating the image data 
# to itself, to form three identical channels

images = np.concatenate((images, images, images), axis=3)
print("Image data shape: {}".format(images.shape))

Image data shape: (200000, 16, 16, 3)


In [ ]:
# Load a reference image
ref_image = plt.imread("../../data/reference_cabin.png")
ref_image = ref_image.reshape((1,)+ref_image.shape)
print(ref_image.shape)

## Pretrained feature extraction


In [ ]:
manual_inspect = False
if manual_inspect:
    # Compare feature output for reference image with a single and double image
    plt.plot(range(len(reference_features[0])), reference_features[0], alpha=0.5, label='reference')
    plt.plot(range(len(single_features[0])), single_features[0], alpha=0.5, label='single')
    plt.plot(range(len(double_features[0])), double_features[0], alpha=0.5, label='double')
    plt.legend()
    plt.show()
    
    # Check distribution of features by inspection
    index = 0 
    fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,12))
    for i in range(3):
        for j in range(3):
            # plot features
            ax[i, j].hist(single_features[:,index + i*3 + j], alpha=0.5, label='single')
            ax[i, j].hist(double_features[:,index + i*3 + j], alpha=0.5, label='double')
            ax[i, j].hist(ref_vgg_features, alpha=0.5, label='reference')
            ax[i, j].legend()
    plt.show()

In [3]:
from scipy.stats import ks_2samp
from joblib import Parallel, delayed
# Check difference using Kolmogorov-Smirnov

def get_pval(i):
    ks = ks_2samp(single_features[:,i], double_features[:,i])
    return ks.pvalue



In [ ]:
# Setup push notifications
from notify_run import Notify
notify = Notify()
notify.register()

In [ ]:
# Run test on all pretrained nets.
p_output = {}

# Keys: model names, Values: depth to compare at.
pretrained_models = {
    #"DenseNet121":None, #8
    #"DenseNet169":None, #8
    #"DenseNet201":None, #8
    #"InceptionResNetV2":None, #8
    #"InceptionV3":None, #8
    #"MobileNet":None, #8
    #"MobileNetV2":None, #5
    #"NASNetLarge":None, #4
    #"NASNetMobile":None, #4
    #"ResNet50":None, #8
    "VGG16":None,
    #"VGG19":None,
    #"Xception":None, #6
    }                       

for net, depth in pretrained_models.items():
    print("Running for:", net)
    # Build net at desired depth
    pretrained = pretrained_model(which_model=net, output_depth=depth)
    
    # Extract features and split them into single and double
    pretrained_features = pretrained.predict(images)
    single_features = pretrained_features[np.where(labels[:,0] == 1)]
    double_features = pretrained_features[np.where(labels[:,1] == 1)]
    n = pretrained_features.shape[1]
    p_values = Parallel(n_jobs=-1, verbose=2)(delayed(get_pval)(i) for i in range(n))
    p_output[net] = p_values
    
    plt.close()
    plt.plot(range(len(p_values)), p_values, label=net)
    plt.legend()
    plt.savefig(net + "-p-vals.png")
    #notify.send(net + " finished!")



In [74]:
print("Running for:", "VGG16")
# Build net at desired depth
pretrained = pretrained_model(which_model="VGG16", output_depth=6)

# Extract features and split them into single and double
pretrained_features = pretrained.predict(images)
single_features = pretrained_features[np.where(labels[:,0] == 1)]
double_features = pretrained_features[np.where(labels[:,1] == 1)]
n = pretrained_features.shape[1]
p_values = Parallel(n_jobs=-1, verbose=2)(delayed(get_pval)(i) for i in range(n))
p_output[net] = p_values

plt.close()
plt.plot(range(len(p_values)), p_values, label=net)
plt.legend()
#plt.savefig(net + "-p-vals.png")

Running for: VGG16


MemoryError: Unable to allocate array with shape (200000, 8192) and data type float32

## Classification with custom dense network
### Build dense model

In [72]:
# Train a fully-connected network to classify based on
# extracted features
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
model = Sequential()
model.add(Dense(512, input_shape=pretrained_features.shape[1:]))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_13 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 1026      
Total params: 526,338
Trainable params: 526,338
Non-trainable params: 0
_________________________________________________________________


### Set up training and test data

In [ ]:
from sklearn.model_selection import train_test_split

# Remove nan values from pretrained_features
nan_indices = []
for i in range(pretrained_features.shape[0]):
    if i%10000 == 0:
        print(i)
    if np.isnan(pretrained_features[i,:]).any():
        nan_indices.append(i)
pretrained_features = np.delete(pretrained_features, nan_indices, axis=0)


# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(pretrained_features, labels, test_size = 0.2)
print("Training and test data shapes:")
print("x_train: {}".format(x_train.shape))
print("x_test: {}".format(x_test.shape))
print("y_train: {}".format(y_train.shape))
print("y_test: {}".format(y_test.shape))

In [73]:
# Train the model
history = model.fit(
    x_train, 
    y_train, 
    epochs=10, 
    batch_size=32,
    validation_data=(x_test, y_test))

Train on 159996 samples, validate on 40000 samples
Epoch 1/10
159996/159996 [==============================] - 23s 144us/sample - loss: 0.3080 - accuracy: 0.8765 - val_loss: nan - val_accuracy: 0.9079
Epoch 2/10
159996/159996 [==============================] - 23s 143us/sample - loss: 0.2685 - accuracy: 0.8976 - val_loss: nan - val_accuracy: 0.9128
Epoch 3/10
159996/159996 [==============================] - 23s 142us/sample - loss: 0.2574 - accuracy: 0.9032 - val_loss: nan - val_accuracy: 0.8996
Epoch 4/10
 85280/159996 [==============>...............] - ETA: 9s - loss: 0.2538 - accuracy: 0.9055

KeyboardInterrupt: 

In [68]:
a = model.predict(x_test[100:101])
print(a)

[[nan nan]]


In [ ]:

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Check predictions
predicted = model.predict(x_test)
print(len(predicted[np.where(predicted[:,1] == 1.0)]))